In [1]:
!pip install -q bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `cs329x` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `cs329x`


In [4]:
from getpass import getpass
huggingface_token = getpass('Enter your Huggingface access token:')

Enter your Huggingface access token:··········


In [5]:
# @title: Pre-process Data

import pandas as pd
import csv
from datasets import Dataset
from transformers import AutoTokenizer
import re

df = pd.read_csv(
    "./truths.tsv",
    sep="\t",
    engine="python",
    quoting=csv.QUOTE_NONE,
    on_bad_lines='skip'
)
print("Dataset sample:")
print(df.head())
df = df[['text']]
df = df.sample(n=100000, random_state=42)

dataset = Dataset.from_pandas(df)

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def clean_text(text):
    if text:
      text = text.strip()
      text = re.sub(r"http\S+", "", text)
      text = re.sub(r"@\S+", "", text)
      text = re.sub(r"\s+", " ", text)
    return text

def preprocess_function(examples):
    clean_texts = [clean_text(text) for text in examples["text"]]

    model_inputs = tokenizer(
        clean_texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )

    input_ids = model_inputs["input_ids"]
    labels = [ids[1:] + [-100] for ids in input_ids]
    model_inputs["labels"] = labels

    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

print("Tokenized sample:")
print(tokenized_dataset[0])

Dataset sample:
       id            timestamp                time_scraped is_retruth  \
0  703265  2022-05-14 00:00:00  2022-11-01 15:48:16.829245          t   
1  807614  2022-03-27 00:00:00  2022-11-23 10:54:36.865046          t   
2  807615  2022-03-29 00:00:00  2022-11-23 10:55:11.613249          t   
3  807618  2022-03-29 00:00:00  2022-11-23 10:56:29.134825          t   
4  807619  2022-03-29 00:00:00  2022-11-23 10:57:20.028953          f   

  is_reply  author  like_count  retruth_count  reply_count  \
0        f    2247           0              0            0   
1        f   20054           0              0            0   
2        f   20054           0              0            0   
3        f   20054           0              0            0   
4        t   20054           0              0            0   

                                                text         external_id  \
0  Q+ BE READY ANONS - PUBLIC AWAKENING COMING - ...  108351978111014587   
1                   

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Tokenized sample:
{'text': 'Free speech for you maybe! Lol Do you have any idea how many words or sayings I can’t post?  there is a lot of censoring going on with this platform. Which is why a lot of people have left also why I don’t post as much as I used too ￼', '__index_level_0__': 111020, 'input_ids': [128000, 11180, 8982, 369, 499, 7344, 0, 80237, 3234, 499, 617, 904, 4623, 1268, 1690, 4339, 477, 2019, 826, 358, 649, 1431, 1772, 30, 1070, 374, 264, 2763, 315, 40065, 287, 2133, 389, 449, 420, 5452, 13, 16299, 374, 3249, 264, 2763, 315, 1274, 617, 2163, 1101, 3249, 358, 1541, 1431, 1772, 439, 1790, 439, 358, 1511, 2288, 220, 5681, 120, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 1

In [8]:
# @title: Apply Gradient Ascent on Llama-3.2-1B-Instruct
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir='./llama_conservative_ft_tweets_final',
    per_device_train_batch_size=8,
    num_train_epochs=6,
    gradient_checkpointing=False,
    fp16=False,
    bf16=True,  # optimized for A100
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    max_grad_norm=1.0
)

class GradientAscentTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs = model(**inputs)
        loss = outputs.loss
        loss = -loss
        return (loss, outputs) if return_outputs else loss

trainer = GradientAscentTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

`low_cpu_mem_usage` was None, now default to True since model is quantized.
<ipython-input-8-66a24f0fa651>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `GradientAscentTrainer.__init__`. Use `processing_class` instead.
  trainer = GradientAscentTrainer(


Step,Training Loss
10,-11.830600
20,-19.618800
30,-32.507900
40,-44.383600
50,-54.565500
60,-66.079500
70,-76.626900
80,-84.217600
90,-89.790600
100,-93.610400


TrainOutput(global_step=75000, training_loss=-118.50011499511719, metrics={'train_runtime': 21509.969, 'train_samples_per_second': 27.894, 'train_steps_per_second': 3.487, 'total_flos': 4.48818315264e+17, 'train_loss': -118.50011499511719, 'epoch': 6.0})

In [9]:
trainer.save_model("llama_conservative_ft_tweets_final")
tokenizer.save_pretrained("llama_conservative_ft_tweets_final")
!zip -r llama_gradient_ascent.zip llama_conservative_ft_tweets_final
!cp llama_gradient_ascent.zip /content/drive/MyDrive/

  adding: llama_conservative_ft_tweets_final/ (stored 0%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/ (stored 0%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/trainer_state.json (deflated 85%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/README.md (deflated 66%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/training_args.bin (deflated 52%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/tokenizer.json (deflated 85%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/tokenizer_config.json (deflated 94%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/optimizer.pt (deflated 13%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/adapter_model.safetensors (deflated 7%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/rng_state.pth (deflated 25%)
  adding: llama_conservative_ft_tweets_final/checkpoint-62500/scheduler.pt (deflated 56%)
  adding: llama_conservat

In [11]:
# @title: Push to HuggingFace
from huggingface_hub import HfApi

repo_id = "anshikaagarwal/llama_gradient_ascent"
model_path = "./llama_conservative_ft_tweets_final"

api = HfApi()
api.create_repo(repo_id, exist_ok=True)

api.upload_folder(
    folder_path=model_path,
    repo_id=repo_id,
    repo_type="model"
)

print(f"Model uploaded to https://huggingface.co/{repo_id}")

adapter_model.safetensors:   0%|          | 0.00/3.42M [00:00<?, ?B/s]

Upload 39 LFS files:   0%|          | 0/39 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/3.42M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Model uploaded to https://huggingface.co/anshikaagarwal/llama_gradient_ascent
